<a href="https://colab.research.google.com/github/Nomaqui/ENTREGAS/blob/main/02%20-%20preprocesado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Leer el contenido, y cargar las librerias que vamos a usar


In [24]:
import pandas as pd
from sklearn.impute import SimpleImputer

df = pd.read_csv('train.csv')

# Guardar columna objetivo y ID
y = df['RENDIMIENTO_GLOBAL']

X = df.drop(columns=['RENDIMIENTO_GLOBAL', 'ID'])


Separar columnas por tipo

In [25]:
cat_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
num_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

print("Categóricas:", cat_cols)
print("Numéricas:", num_cols)

Categóricas: ['ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO', 'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA', 'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE', 'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL', 'ESTU_PRIVADO_LIBERTAD', 'ESTU_PAGOMATRICULAPROPIO', 'FAMI_TIENECOMPUTADOR', 'FAMI_TIENEINTERNET.1', 'FAMI_EDUCACIONMADRE']
Numéricas: ['PERIODO', 'coef_1', 'coef_2', 'coef_3', 'coef_4']


Imputar numéricas con la media

In [26]:
imp_num = SimpleImputer(strategy='mean')
X[num_cols] = imp_num.fit_transform(X[num_cols])

Imputar categóricas con la moda

In [27]:
imp_cat = SimpleImputer(strategy='most_frequent')
X[cat_cols] = imp_cat.fit_transform(X[cat_cols])

Mostrar la tabla con los datos imputados

Convertir columnas en one-hot

In [28]:
X_convertida = pd.get_dummies(X, columns=cat_cols, drop_first=True)

Mostrar la tabla con las columnas debidas en one-hot

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X_convertida, y,  test_size=0.42857184, random_state=19, stratify=y)
clf = RandomForestClassifier(
    n_estimators=400,

    # Más árboles suelen mejorar el rendimiento
    max_depth=None,           # Limita profundidad para evitar overfitting
    min_samples_split=5,      # Requiere más muestras para dividir
    min_samples_leaf=2,       # Reduce overfitting
    max_features='sqrt',      # Usa solo raíz de variables en cada split
    random_state=19
)
clf.fit(X_train, y_train)

In [44]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

        alto       0.50      0.63      0.56     75266
        bajo       0.42      0.57      0.48     74137
  medio-alto       0.32      0.20      0.25     73551
  medio-bajo       0.32      0.24      0.27     73832

    accuracy                           0.41    296786
   macro avg       0.39      0.41      0.39    296786
weighted avg       0.39      0.41      0.39    296786

[[47317  8976 11368  7605]
 [ 8985 41989  8654 14509]
 [23885 19492 15000 15174]
 [14201 29463 12586 17582]]


In [31]:
df_test = pd.read_csv("test.csv")
ids_test = df_test["ID"]
X_test = df_test.drop(columns=["ID"])

In [32]:
# Imputar valores faltantes
X_test[num_cols] = imp_num.transform(X_test[num_cols])
X_test[cat_cols] = imp_cat.transform(X_test[cat_cols])

# Codificación one-hot con las mismas columnas que el entrenamiento
X_test_convertida = pd.get_dummies(X_test, columns=cat_cols, drop_first=True)

# Asegurar que tenga las mismas columnas que X_convertida
X_test_convertida = X_test_convertida.reindex(columns=X_convertida.columns, fill_value=0)

In [33]:
y_test_pred = clf.predict(X_test_convertida)

In [34]:
print(len(y_test_pred))

296786


In [35]:
submission = pd.DataFrame({
    "ID": ids_test,
    "RENDIMIENTO_GLOBAL": y_test_pred
})

In [36]:
submission.to_csv("submission.csv", index=False)
